# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Examine the database

In [2]:
# Connect to SQLite database.

conn = sqlite3.connect('data/prod_sample.db')
cursor = conn.cursor()

# Explore all tables in database

In [3]:
# List all the tables in the database.
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
tables= cursor.execute(sql_query).fetchall()
table_names = ", ".join([table[0] for table in tables])
table_names


'stock_description, online_retail_history'

# Read data from the `online_retail_history` table

In [4]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT * FROM online_retail_history'


# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history = pd.read_sql(query, conn)


# Preview the first five rows of the data.
online_retail_history.head(5)

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [5]:
# Get the shape of the data.
online_retail_history.shape

(15321, 8)

# Read data from the `stock_description` table

In [6]:
# Write the query to be executed that selects everything from the stock_description table.
query = 'SELECT * FROM stock_description'

# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description = pd.read_sql(query, conn)

# Preview the first five rows of the data.

stock_description.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [7]:
stock_description.shape

(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [8]:
query = '''SELECT online_retail_history.*, stock_description.*
           FROM online_retail_history
           LEFT JOIN stock_description ON online_retail_history.StockCode = stock_description.StockCode'''
online_retail_history_agg = pd.read_sql(query, conn)

online_retail_history_agg.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [9]:
# Get the shape of the data.
online_retail_history_agg.shape

(17032, 10)

# Identify and fix corrupt or unusable data

In [10]:
# Check the value counts of the "Description" field.

df = online_retail_history_agg['Description'].count()
df= online_retail_history_agg['Description'].value_counts()
print(df.shape)
df


(11,)


CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
REGENCY CAKESTAND 3 TIER              1711
?                                     1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [11]:
# Remove rows where "Description" is just a question mark (?).
df = online_retail_history_agg[(online_retail_history_agg["Description"] != "?")]
# df= df[["StockCode","Description"]]
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [12]:
df.shape

(15321, 10)

# Identify and remove duplicates

In [13]:
# Identify all duplicated data.
duplicated_data = df[df.duplicated(keep = False)]

print('Number of rows with duplicated data:', duplicated_data.shape[0])

Number of rows with duplicated data: 223


In [14]:
# Print the duplicated data.
duplicated_data.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
176,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,20727,LUNCH BAG BLACK SKULL.
178,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,20727,LUNCH BAG BLACK SKULL.
496,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,84879,ASSORTED COLOUR BIRD ORNAMENT
497,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,84879,ASSORTED COLOUR BIRD ORNAMENT
569,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT


In [15]:
# Remove the duplicated data.

online_retail_history_agg_final = df[~df.duplicated()]

online_retail_history_agg_final.head()



,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [16]:
online_retail_history_agg_final.dtypes

Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
StockCode       object
Description     object
dtype: object

# Correct date formats

In [17]:
# Get the data types for every column in the DataFrame.

online_retail_history_agg_final.dtypes

Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
StockCode       object
Description     object
dtype: object

In [18]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.

# online_retail_history_agg_final = online_retail_history_agg_final.copy()  # Work with a new object.
online_retail_history_agg_final['InvoiceDate'] = pd.to_datetime(online_retail_history_agg_final['InvoiceDate'], format = '%Y-%m-%d')
online_retail_history_agg_final.head()

<ipython-input-18-748b51c1e57d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  online_retail_history_agg_final['InvoiceDate'] = pd.to_datetime(online_retail_history_agg_final['InvoiceDate'], format = '%Y-%m-%d')


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [19]:
# Get the data types for every column in the converted DataFrame.
online_retail_history_agg_final.dtypes

Invoice                object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
CustomerID             object
Country                object
TotalAmount           float64
StockCode              object
Description            object
dtype: object

# Examine the table before finishing

In [20]:
# Preview the first five rows of the data.
online_retail_history_agg_final.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [21]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.

online_retail_history_agg_final.to_pickle('online_history_cleaned.pickle')


# Confirm that the data was written to the pickle file


In [22]:
pd.read_pickle('online_history_cleaned.pickle').head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [23]:
pd.read_pickle('online_history_cleaned.pickle').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   StockCode    15206 non-null  object        
 9   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 950.4+ KB


In [24]:
# Close any connections to the database.
conn.close()


# Summarize software libraries used.

In [25]:
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.8.5 (tags/v3.8.5:580fbb0, Jul 20 2020, 15:43:08) [MSC v.1926 32 bit (Intel)]
- pandas 1.2.4
- sqlite3 3.31.1
